In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = 'True'

In [2]:
!pip install tensorflow==2.11 keras==2.11
!pip install ktrain

  Using cached tensorflow-2.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
  Using cached tensorboard-2.11.2-py3-none-any.whl.metadata (1.9 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl.metadata (1.1 kB)
Using cached tensorflow-2.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-ser

In [3]:
!pip install tf_keras scikit-learn wordcloud ktrain tensorflow
!pip install --upgrade ktrain
!pip install Keras-Preprocessing
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
import ktrain
from ktrain import text
sns.set()
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached protobuf-4.25.4-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.5.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
Using cached keras-3.5.0-py3-none-any.whl (1.1 MB)
Using cached protobuf-4.25.4-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl (6.6 MB)
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-da

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
df=pd.read_csv("processedData.csv")
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [5]:
num_species = df['species_group'].nunique()
num_species

12

In [6]:
df['tail']=df['tail'].map({'yes':1, 'no':0})

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_message = vectorizer.fit_transform(df['processedMessage'])

In [8]:
from scipy.sparse import hstack

X_features = hstack([X_message, np.array(df[['fingers', 'tail']])])
y = df['species']

In [10]:
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['species_number'] = label_encoder.fit_transform(df['species_group'])

X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.15, random_state=19)

## Random Forest

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Function to combine labels
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced')

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_val)

y_pred4 = combine_labels(pd.Series(y_pred))
y_val_combined = combine_labels(pd.Series(y_val))

accuracy = accuracy_score(y_val_combined, y_pred4)
print(f'Validation Accuracy: {accuracy}')

print(classification_report(y_val_combined, y_pred4))


Validation Accuracy: 0.8666666666666667
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      1.00      1.00         9
    Emotivor       0.88      1.00      0.93         7
      Faerix       1.00      0.71      0.83         7
     Florian       1.00      0.73      0.84        11
     Mythron       0.90      1.00      0.95         9
      Nexoon       1.00      0.88      0.93         8
     Quixnar       0.50      0.75      0.60         4
     Sentire       1.00      1.00      1.00         8
     Zorblax       0.67      0.57      0.62         7

    accuracy                           0.87        75
   macro avg       0.86      0.86      0.85        75
weighted avg       0.89      0.87      0.87        75



## Logistic Regression

In [42]:
lr = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred1)))
print(classification_report(y_val, y_pred1))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Accuracy: 0.8666666666666667
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      1.00      1.00         9
    Emotivor       0.75      0.86      0.80         7
      Faerix       0.86      0.86      0.86         7
     Florian       1.00      0.64      0.78        11
     Mythron       0.80      0.89      0.84         9
      Nexoon       1.00      0.88      0.93         8
     Quixnar       0.80      1.00      0.89         4
     Sentire       0.88      0.88      0.88         8
     Zorblax       1.00      0.86      0.92         7

    accuracy                           0.87        75
   macro avg       0.87      0.88      0.87        75
weighted avg       0.89      0.87      0.87        75



## SVM (Support Vector Machines)

In [43]:
svc = LinearSVC()
svc.fit(X_train, y_train)
y_pred2 = svc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred2)))
print(classification_report(y_val, y_pred2))

Accuracy: 0.8533333333333334
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      1.00      1.00         9
    Emotivor       0.75      0.86      0.80         7
      Faerix       0.86      0.86      0.86         7
     Florian       1.00      0.73      0.84        11
     Mythron       0.80      0.89      0.84         9
      Nexoon       1.00      0.88      0.93         8
     Quixnar       0.67      1.00      0.80         4
     Sentire       0.86      0.75      0.80         8
     Zorblax       1.00      0.71      0.83         7

    accuracy                           0.85        75
   macro avg       0.86      0.87      0.85        75
weighted avg       0.88      0.85      0.86        75



/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Naive Bayes

In [44]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred3 = mnb.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred3)))
print(classification_report(y_val, y_pred3))

Accuracy: 0.5733333333333334
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      0.33      0.50         9
    Emotivor       1.00      0.14      0.25         7
      Faerix       0.00      0.00      0.00         7
     Florian       1.00      0.55      0.71        11
     Mythron       0.56      1.00      0.72         9
      Nexoon       0.50      0.50      0.50         8
     Quixnar       0.38      0.75      0.50         4
     Sentire       0.57      1.00      0.73         8
     Zorblax       0.36      0.57      0.44         7

    accuracy                           0.57        75
   macro avg       0.60      0.58      0.51        75
weighted avg       0.64      0.57      0.52        75



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gradient Boost

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define Gradient Boosting model with valid max_features
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', max_depth=4, random_state=1, verbose=1)

gbc.fit(X_train, y_train)
y_pred5 = gbc.predict(X_val)
print(accuracy_score(y_val, y_pred5))
print(classification_report(y_val, y_pred5))


      Iter       Train Loss   Remaining Time 
         1           2.0111            5.25s
         2           1.8456            4.35s
         3           1.6989            4.54s
         4           1.5627            4.62s
         5           1.4557            4.60s
         6           1.3599            4.89s
         7           1.2778            5.06s
         8           1.1975            5.02s
         9           1.1372            5.34s
        10           1.0722            5.13s
        20           0.6443            4.39s
        30           0.4217            3.43s
        40           0.2893            2.63s
        50           0.2093            2.26s
        60           0.1500            1.85s
        70           0.1099            1.37s
        80           0.0832            0.86s
        90           0.0631            0.41s
       100           0.0485            0.00s
0.7466666666666667
              precision    recall  f1-score   support

      Aquari       0.50  

## Ensemble/Voting Classifier

In [72]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Function to combine labels
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

# Custom classifier that integrates RandomForestClassifier and label combining
class CustomRandomForestClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=300, max_depth=15, random_state=42, class_weight='balanced'):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.class_weight = class_weight
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)

    def fit(self, X, y):
        self.rfc.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.rfc.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.rfc.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
            'random_state': self.random_state,
            'class_weight': self.class_weight
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)
        return self

custom_rfc = CustomRandomForestClassifier(n_estimators=300, max_depth=15, random_state=42, class_weight='balanced')

class CustomLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, C=2, max_iter=1000, n_jobs=-1):
        self.C = C
        self.max_iter = max_iter
        self.n_jobs = n_jobs
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)

    def fit(self, X, y):
        self.lr.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.lr.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.lr.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'C': self.C,
            'max_iter': self.max_iter,
            'n_jobs': self.n_jobs
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)
        return self

custom_lr = CustomLogisticRegression(C=2, max_iter=1000, n_jobs=-1)

mnb = MultinomialNB()
rfc = RandomForestClassifier(n_estimators=1000, max_depth=12, random_state=42)
lr = LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
svc = SVC(probability=True)

# Create the VotingClassifier with your custom classifier
ec = VotingClassifier(
    estimators=[
        ('Multinominal NB', mnb),
        ('Custom Random Forest', custom_rfc),
        ('Custom Logistic Regression', custom_lr),
        ('Support Vector Machine', svc)
    ],
    voting='hard',
    weights=[50, 100, 100, 90]
)

ec.fit(X_train, y_train)

y_pred6 = ec.predict(X_val)
print(accuracy_score(y_val, y_pred6))
print(classification_report(y_val, y_pred6))


0.88
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      1.00      1.00         9
    Emotivor       0.88      1.00      0.93         7
      Faerix       1.00      0.71      0.83         7
     Florian       1.00      0.73      0.84        11
     Mythron       0.90      1.00      0.95         9
      Nexoon       1.00      0.88      0.93         8
     Quixnar       0.60      0.75      0.67         4
     Sentire       1.00      1.00      1.00         8
     Zorblax       0.71      0.71      0.71         7

    accuracy                           0.88        75
   macro avg       0.87      0.88      0.86        75
weighted avg       0.90      0.88      0.88        75



## Random Forest With AdaBoost

In [56]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=9, random_state=0)
abc= AdaBoostClassifier(base_estimator=rfc, learning_rate=0.2, n_estimators=100)
abc.fit(X_train, y_train)
y_pred7= abc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val, y_pred7)))
print(classification_report(y_val, y_pred7))

Accuracy: 0.88
              precision    recall  f1-score   support

      Aquari       0.62      1.00      0.77         5
       Cybex       1.00      1.00      1.00         9
    Emotivor       1.00      1.00      1.00         7
      Faerix       1.00      0.86      0.92         7
     Florian       1.00      0.73      0.84        11
     Mythron       0.90      1.00      0.95         9
      Nexoon       1.00      1.00      1.00         8
     Quixnar       0.43      0.75      0.55         4
     Sentire       1.00      1.00      1.00         8
     Zorblax       0.75      0.43      0.55         7

    accuracy                           0.88        75
   macro avg       0.87      0.88      0.86        75
weighted avg       0.91      0.88      0.88        75



## Comparision Between Different Models

In [57]:
Comparison_unibi = pd.DataFrame({'Logistic Regression': [accuracy_score(y_val,y_pred1)*100,f1_score(y_val,y_pred1,average='macro')*100,recall_score(y_val, y_pred1,average='micro')*100,precision_score(y_val, y_pred1,average='micro')*100],
                            'SVM':[accuracy_score(y_val,y_pred2)*100,f1_score(y_val,y_pred2,average='macro')*100,recall_score(y_val, y_pred2,average='micro')*100,precision_score(y_val, y_pred2,average='micro')*100],
                           'Naive Bayes':[accuracy_score(y_val,y_pred3)*100,f1_score(y_val,y_pred3,average='macro')*100,recall_score(y_val, y_pred3,average='micro')*100,precision_score(y_val, y_pred3,average='micro')*100],
                           'Random Forest':[accuracy_score(y_val,y_pred4)*100,f1_score(y_val,y_pred4,average='macro')*100,recall_score(y_val, y_pred4,average='micro')*100,precision_score(y_val, y_pred4,average='micro')*100],
                           'GradientBoosting':[accuracy_score(y_val,y_pred5)*100,f1_score(y_val,y_pred5,average='macro')*100,recall_score(y_val, y_pred5,average='micro')*100,precision_score(y_val, y_pred5,average='micro')*100],
                           'Ensembled':[accuracy_score(y_val,y_pred6)*100,f1_score(y_val,y_pred6,average='macro')*100,recall_score(y_val, y_pred6,average='micro')*100,precision_score(y_val, y_pred6,average='micro')*100],
                           'Adaboost':[accuracy_score(y_val,y_pred7)*100,f1_score(y_val,y_pred7,average='macro')*100,recall_score(y_val, y_pred7,average='micro')*100,precision_score(y_val, y_pred7,average='micro')*100],

})

In [59]:
Comparison_unibi.rename(index={0:'Accuracy',1:'F1_score', 2: 'Recall',3:'Precision'}, inplace=True)
Comparison_unibi.head()

,Logistic Regression,SVM,Naive Bayes,Random Forest,GradientBoosting,Ensembled,Adaboost
Accuracy,86.666667,85.333333,57.333333,86.666667,74.666667,88.000000,88.000000
F1_score,86.665558,84.772508,51.168303,84.740891,73.072051,86.396568,85.726905
Recall,86.666667,85.333333,57.333333,86.666667,74.666667,88.000000,88.000000
Precision,86.666667,85.333333,57.333333,86.666667,74.666667,88.000000,88.000000
